ランダムフォレストによる因果推論

これまで線形和の関係性のみを扱っていたが，機械学習による推論部分についてひ

# 自前データの作成

In [1]:
# 乱数シード
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)


# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *
# グラフの描画用
import matplotlib.pyplot as plt
# SciPy 平均0、分散1に正規化（標準化）関数
import scipy.stats
# シグモイド関数をimport
from scipy.special import expit
# その他
import pandas as pd


# サンプル数
num_data = 200

# 性別 / 年齢（一様分布）
# age
x_1 = randint(15, 76, num_data) 
# 性別（0: 女性，1: 男性）
x_2 = randint(0, 2, num_data)


# テレビCMをみたかどうか
# ノイズの生成
e_z = randn(num_data)
# シグモイド関数に入れる線形関数・・・（1）
z_base = x_1 + (1 - x_2)*10 - 40 + 5*e_z
# シグモイド関数の計算
z_prob = expit(0.1*z_base)
# テレビCMを見たかどうか（0: 見ていない，1: 見た）
Z = np.array([])
# （1）より年配の女性の方が見やすくなる
for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)
    

# 購入量Y
# ノイズの生成
e_y = randn(num_data)
# 若年層で，男性で，テレビを見ていると購入が多くなる
Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y 

# 作成データ

In [2]:
df = pd.DataFrame({
    'age': x_1,
    'sex': x_2,
    'cm': Z,
    'amount': Y
})

In [3]:
df.head()

,age,sex,cm,amount
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


## CM別平均

In [4]:
print(df[df['cm']==1.0].mean())
print('------')
print(df[df['cm']==0.0].mean())

age       55.836066
sex        0.483607
cm         1.000000
amount    49.711478
dtype: float64
------
age       32.141026
sex        0.692308
cm         0.000000
amount    68.827143
dtype: float64


# 決定木による分類

決定木でCMを見たかどうかを分類予測するモデルを作成する

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# 説明変数
X = df[['age', 'sex']]

# 目的変数
Z = df['cm']

# train / test
X_train, X_val, Z_train, Z_val = train_test_split(X, Z, train_size=0.6, random_state=0)

# 学習と性能確認
clf = DecisionTreeClassifier(max_depth=1, random_state=0)
clf.fit(X_train, Z_train)
print('深さ1の性能:', clf.score(X_val, Z_val)) # 正解率
clf = DecisionTreeClassifier(max_depth=2, random_state=0)
clf.fit(X_train, Z_train)
print('深さ2の性能:', clf.score(X_val, Z_val)) # 正解率
clf = DecisionTreeClassifier(max_depth=3, random_state=0)
clf.fit(X_train, Z_train)
print('深さ3の性能:', clf.score(X_val, Z_val)) # 正解率

深さ1の性能: 0.85
深さ2の性能: 0.85
深さ3の性能: 0.825


# 決定木による回帰

決定木で購入量を回帰予測するモデルを作成する

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

# 説明変数
X = df[['age', 'sex']]

# 目的変数
Y = df['amount']

# train / test
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, train_size=0.6, random_state=0)

# 学習と性能確認
reg = DecisionTreeRegressor(max_depth=2, random_state=0)
reg.fit(X_train, Y_train)
print('深さ1の性能:', reg.score(X_val, Y_val)) # 決定係数R
reg = DecisionTreeRegressor(max_depth=3, random_state=0)
reg.fit(X_train, Y_train)
print('深さ2の性能:', reg.score(X_val, Y_val)) # 決定係数R
reg = DecisionTreeRegressor(max_depth=4, random_state=0)
reg.fit(X_train, Y_train)
print('深さ3の性能:', reg.score(X_val, Y_val)) # 決定係数R

深さ1の性能: 0.7257496664596153
深さ2の性能: 0.7399348963931736
深さ3の性能: 0.7165539691159017


# ランダムフォレストによる分類

ランダムフォレスト でCMを見たかどうかを分類予測するモデルを作成する

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# 説明変数
X = df[['age', 'sex']]

# 目的変数
Z = df['cm']

# train / test
X_train, X_val, Z_train, Z_val = train_test_split(X, Z, train_size=0.6, random_state=0)

# 学習と性能確認
clf = RandomForestClassifier(max_depth=1, random_state=0)
clf.fit(X_train, Z_train)
print('深さ1の性能:', clf.score(X_val, Z_val)) # 正解率
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, Z_train)
print('深さ2の性能:', clf.score(X_val, Z_val)) # 正解率
clf = RandomForestClassifier(max_depth=3, random_state=0)
clf.fit(X_train, Z_train)
print('深さ3の性能:', clf.score(X_val, Z_val)) # 正解率

深さ1の性能: 0.775
深さ2の性能: 0.85
深さ3の性能: 0.825


# ランダムフォレストによる回帰

ランダムフォレスト購入量を回帰予測するモデルを作成する

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# 説明変数
X = df[['age', 'sex']]

# 目的変数
Y = df['amount']

# train / test
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, train_size=0.6, random_state=0)

# 学習と性能確認
reg = RandomForestRegressor(max_depth=2, random_state=0)
reg.fit(X_train, Y_train)
print('深さ1の性能:', reg.score(X_val, Y_val)) # 決定係数R
reg = RandomForestRegressor(max_depth=3, random_state=0)
reg.fit(X_train, Y_train)
print('深さ2の性能:', reg.score(X_val, Y_val)) # 決定係数R
reg = RandomForestRegressor(max_depth=4, random_state=0)
reg.fit(X_train, Y_train)
print('深さ3の性能:', reg.score(X_val, Y_val)) # 決定係数R

深さ1の性能: 0.7618786062003249
深さ2の性能: 0.7810610687821997
深さ3の性能: 0.7655149049335737
